# Project 1

In [1]:
from random import *
from time import *
from math import *
from copy import *
from collections import *
from enum import *

### PriorityQueue class

In [2]:
import heapq
class PriorityQueue:
    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = []
        for item in items:
            self.add(item)
    def __len__(self): return len(self.items)
    def top(self): return self.items[0][1]
    def add(self, item):
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)
    def pop(self):
        # Pop and return the item with min f(item) value.
        return heapq.heappop(self.items)[1]


In [3]:
def map_input(file_name):
    grid = []
    with open(file_name, 'r') as file:
        
        n, m = map(int, file.readline().split(','))
        while floorname := file.readline().strip():
            floor = []
            for i in range(n):
                floor.append(list(file.readline().strip().split(',')))
            grid.append(floor)

    return grid
grid = map_input("test3.txt")
print(grid)


[[['-1', 'K1', '0', 'A1', '0'], ['0', '0', '0', '-1', '0'], ['-1', '-1', '-1', '0', '0'], ['0', 'UP', '0', 'D1', '0'], ['-1', '0', '0', 'D1', '0']], [['-1', '0', '0', '0', '0'], ['0', '0', '0', '-1', '0'], ['-1', '-1', '-1', '0', '0'], ['0', '0', '0', '0', '0'], ['-1', 'DO', '0', '0', 'T1']]]


#### PriorityQueue test

In [4]:
pq = PriorityQueue([1,2,3,4,5])
print(pq.pop())
print(pq.pop())
print(pq.pop())
print(pq.pop())
print(pq.pop())
print()
pq = PriorityQueue([1,2,3,4,5], key=lambda x: -x)
print(pq.pop())
print(pq.pop())
print(pq.pop())
print(pq.pop())
print(pq.pop())
print()

1
2
3
4
5

5
4
3
2
1



## Level 1

### GUI

In [5]:
import tkinter as tk
import tkinter.ttk as ttk

In [6]:
class MapGUI:
    WINDOW_WIDTH = 1366
    WINDOW_HEIGHT = 768
    PANEL_WIDTH = 400
    COLOR_BLOCKED = "#3f3f3f"
    COLOR_EMPTY = "lightgray"
    COLOR_DOOR = "burlywood4"
    COLOR_KEY = "yellow"
    COLOR_START = "brown1"
    TCOLOR_START = "black"
    COLOR_TARGET = "chartreuse3"
    TCOLOR_TARGET = "black"
    COLOR_CURRENT = "blue"
    COLOR_PATH = "lightblue"
    TCOLOR_PATH = "red"

    def __init__(self, root, grid, paths, starts, targets):
        # initialize the number of floor, the width and height of the grid
        self.floors = len(grid)
        self.num_rows = len(grid[0])
        self.num_columns = len(grid[0][0])

        self.CELL_SIZE = min(MapGUI.WINDOW_HEIGHT // self.num_rows,
                             (MapGUI.WINDOW_WIDTH - MapGUI.PANEL_WIDTH) // self.num_columns,)

        # tkinter gui handler
        self.root = root

        # initialize information

        self.grid = grid
        self.paths = paths

        self.num_agents = len(self.paths)
        self.current_agent_path_indexes = [0] * self.num_agents
        self.current_agent_turn = 0
        self.current_floor = self.paths[self.current_agent_turn][self.current_agent_path_indexes[self.current_agent_turn]][0]

        self.starts = starts
        self.targets = targets

        self.canvas = tk.Canvas(self.root, width=self.CELL_SIZE * self.num_columns, height=self.CELL_SIZE * self.num_rows)
        self.canvas.pack(side=tk.LEFT)

        self.panel = tk.Frame(self.root, width=MapGUI.PANEL_WIDTH, height=MapGUI.WINDOW_HEIGHT, highlightbackground="blue", highlightthickness=1)
        self.panel.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        self.current_position_var = tk.StringVar()
        self.current_point_var = tk.StringVar()
        self.current_floor_var = tk.StringVar()

        # Labels to display information, make it align left with some margin
        tk.Label(self.panel, text="Current Position:").grid(row=0, column=0)
        tk.Label(self.panel, textvariable=self.current_position_var).grid(row=0, column=1)
        tk.Label(self.panel, text="Current Point:").grid(row=1, column=0)
        tk.Label(self.panel, textvariable=self.current_point_var).grid(row=1, column=1)
        tk.Label(self.panel, text="Current Floor:").grid(row=2, column=0)
        tk.Label(self.panel, textvariable=self.current_floor_var).grid(row=2, column=1)
        
        # spacer in between the informations and the buttons
        tk.Label(self.panel).grid(row=3, column=0)
        
        # Buttons
        tk.Button(self.panel, text="Previous", command=self.to_previous).grid(row=4, column=0)
        tk.Button(self.panel, text="Next", command=self.to_next).grid(row=4, column=1)

        # is_looping
        self.is_looping = True

        self.setup()
        self.draw_grid()
        self.root.mainloop()

    def update_labels(self):
        # change label
        self.current_position_var.set(str(self.paths[self.current_agent_turn][self.current_agent_path_indexes[self.current_agent_turn]]))
        self.current_floor_var.set(str(self.paths[self.current_agent_turn][self.current_agent_path_indexes[self.current_agent_turn]][0]))

    def to_start(self):
        # update the current position
        self.current_agent_path_indexes = [0] * self.num_agents
        self.current_agent_turn = 0
        self.current_floor = self.paths[self.current_agent_turn][self.current_agent_path_indexes[self.current_agent_turn]][0]
        
        # update the labels
        self.update_labels()
        
        self.draw_grid()

    # def to_end(self):
    #     # update the current position
    #     self.current_agent_path_indexes = [len(path) - 1 for path in self.paths]
    #     self.current_agent_turn = self.num_agents - 1
    #     self.current_floor = self.paths[self.current_agent_turn][self.current_agent_path_indexes[self.current_agent_turn]][0]

    #     # update the labels
    #     self.update_labels()

    #     self.draw_grid()

    def to_previous(self, redraw=True):
        # update the current position
        if self.current_agent_path_indexes[self.current_agent_turn] - 1 < 0:
            return
        
        self.current_agent_path_indexes[self.current_agent_turn] -= 1
        
        self.current_agent_turn = (self.current_agent_turn - 1) % self.num_agents

        self.current_floor = self.paths[self.current_agent_turn][self.current_agent_path_indexes[self.current_agent_turn]][0]

        # update the labels
        self.update_labels()

        self.draw_grid()

        
    def to_next(self, redraw=True):
        # update the current position
        if self.current_agent_path_indexes[self.current_agent_turn] + 1 >= len(self.paths[self.current_agent_turn]):
            return
        
        self.current_agent_path_indexes[self.current_agent_turn] += 1
        
        self.current_agent_turn = (self.current_agent_turn + 1) % self.num_agents

        self.current_floor = self.paths[self.current_agent_turn][self.current_agent_path_indexes[self.current_agent_turn]][0]

        # update the labels
        self.update_labels()

        self.draw_grid()


    def setup(self):
        self.root.title("CS420-Project1")
        self.root.geometry(str(MapGUI.WINDOW_WIDTH) + "x" + str(MapGUI.WINDOW_HEIGHT))
        self.root.resizable(False, False)
        self.root.after(500, self.loop)

    def loop(self):
        if self.is_looping:
            self.to_next()
        self.root.after(500, self.loop)

    def draw_grid(self, heatmap=False):
        if heatmap:
            self.color_grids()
            self.draw_heatmap()
        else:
            self.color_grids()
            self.draw_path()
            self.color_current_position()
            self.color_starts()
            self.color_targets()

    def color_grids(self):
        for row in range(self.num_rows):
            for col in range(self.num_columns):
                cell_value = self.grid[self.current_floor][row][col]
                if cell_value == '-1':
                    self.draw_cell(row, col, background_fill=MapGUI.COLOR_BLOCKED)
                elif cell_value == '0':
                    self.draw_cell(row, col, background_fill=MapGUI.COLOR_EMPTY)
                elif cell_value.startswith('K'):
                    self.draw_cell(row, col, background_fill=MapGUI.COLOR_KEY, text=cell_value)
                elif cell_value.startswith('D'):
                    self.draw_cell(row, col, background_fill=MapGUI.COLOR_DOOR, text=cell_value)

    def draw_path(self):
        # draw the path in the current floor
        for turn, path in enumerate(self.paths):
            for i in range(self.current_agent_path_indexes[turn]):
                # in the current floor entirely
                if path[i][0] == self.current_floor and path[i + 1][0] == self.current_floor:
                    self.canvas.create_line(path[i][2] * self.CELL_SIZE + self.CELL_SIZE / 2,
                                            path[i][1] * self.CELL_SIZE + self.CELL_SIZE / 2,
                                            path[i + 1][2] * self.CELL_SIZE + self.CELL_SIZE / 2,
                                            path[i + 1][1] * self.CELL_SIZE + self.CELL_SIZE / 2,
                                            fill=MapGUI.TCOLOR_PATH, width=3)

    def color_current_position(self):
        for path in self.paths:
            if len(path) <= 0:
                continue

            current_path_index = self.current_agent_path_indexes[self.current_agent_turn]
            
            if path[current_path_index][0] == self.current_floor:
                self.draw_cell(path[current_path_index][1], path[current_path_index][2], background_fill=MapGUI.COLOR_CURRENT)

    def color_starts(self):
        for i, start in enumerate(list(filter(lambda x: x[0] == self.current_floor, self.starts))):
            self.draw_cell(start[1], start[2], text="A" + str(i + 1), background_fill=MapGUI.COLOR_START, text_fill=MapGUI.TCOLOR_START)

    def color_targets(self):
        for i, target in enumerate(list(filter(lambda x: x[0] == self.current_floor, self.targets))):
            self.draw_cell(target[1], target[2], text="T" + str(i + 1), background_fill=MapGUI.COLOR_TARGET, text_fill=MapGUI.TCOLOR_TARGET)


    def draw_cell(self, row, col, text="", background_fill=None, text_fill="black"):
        x0 = col * self.CELL_SIZE
        y0 = row * self.CELL_SIZE
        x1 = x0 + self.CELL_SIZE
        y1 = y0 + self.CELL_SIZE
        self.canvas.create_rectangle(x0, y0, x1, y1, outline="black", fill=background_fill)

        # draw text in the center of the cell
        x = (x0 + x1) / 2
        y = (y0 + y1) / 2
        self.canvas.create_text(x, y, text=text, font=("Arial", int(self.CELL_SIZE / 2.5), 'bold'), fill=text_fill)
    

In [7]:
grid = map_input("test2.txt")[0]

In [8]:
def get_starts(grid):
    starts = []
    for k in range(len(grid)):
        for i in range(len(grid[0])):
            for j in range(len(grid[0][0])):
                if grid[k][i][j].startswith("A"):
                    starts.append((k, i, j))
    return starts

In [9]:
def get_targets(grid):
    targets = []
    for k in range(len(grid)):
        for i in range(len(grid[0])):
            for j in range(len(grid[0][0])):
                if grid[k][i][j].startswith("T"):
                    targets.append((k, i, j))
    return targets

### BFS

In [10]:
from collections import deque

class BFS:
    def __init__(self, grid):
        self.grid = grid
        self.n = len(grid)
        self.m = len(grid[0])
        self.explored = [[False] * self.m for _ in range(self.n)]
        self.directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        self.diagonals = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        self.path = [[(-1, -1)] * self.m for _ in range(self.n)]
        for i in range(self.n):
            for j in range(self.m):
                if grid[i][j] == "A1":
                    self.start = (i, j)
                elif grid[i][j] == "T1":
                    self.target = (i, j)

    def is_valid(self, x, y):
        return 0 <= x < self.n and 0 <= y < self.m and self.grid[x][y] != '-1'

    def process(self):
        queue = deque([(self.start[0], self.start[1])])
        self.explored[self.start[0]][self.start[1]] = True

        while queue:
            x, y = queue.popleft()

            if (x, y) == self.target:
                return True

            for dx, dy in self.directions:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny]:
                    queue.append((nx, ny))
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
                    
            for dx, dy in self.diagonals:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny] and self.is_valid(x + dx, y) and self.is_valid(x, y + dy):
                    queue.append((nx, ny))
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
        return False
    
    def get_path(self):
        path = []
        x, y = self.target
        while (x, y) != (-1, -1):
            path.append((x, y))
            x, y = self.path[x][y]
        return path[::-1]

In [11]:
grid = map_input("test2.txt")
bfs = BFS(grid[0])
bfs.process()
path = bfs.get_path()
paths = [[(0, x, y) for x, y in path]]
starts = get_starts(grid)
targets = get_targets(grid)
gui = MapGUI(tk.Tk(), grid, paths, starts, targets)

### DFS

In [12]:
class DFS:
    def __init__(self, grid):
        self.grid = grid
        self.n = len(grid)
        self.m = len(grid[0])
        self.explored = [[False] * self.m for _ in range(self.n)]
        self.directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        self.diagonals = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        self.path = [[(-1, -1)] * self.m for _ in range(self.n)]
        for i in range(self.n):
            for j in range(self.m):
                if grid[i][j] == "A1":
                    self.start = (i, j)
                elif grid[i][j] == "T1":
                    self.target = (i, j)
                    
    def is_valid(self, x, y):
        return 0 <= x < self.n and 0 <= y < self.m and self.grid[x][y] != '-1'
    
    def process(self):
        stack = [(self.start[0], self.start[1])]
        self.explored[self.start[0]][self.start[1]] = True

        while stack:
            x, y = stack.pop()

            if (x, y) == self.target:
                return True

            for dx, dy in self.directions:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny]:
                    stack.append((nx, ny))
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
                    
            for dx, dy in self.diagonals:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny] and self.is_valid(x + dx, y) and self.is_valid(x, y + dy):
                    stack.append((nx, ny))
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
        return False
    
    def get_path(self):
        path = []
        x, y = self.target
        while (x, y) != (-1, -1):
            path.append((x, y))
            x, y = self.path[x][y]
        return path[::-1]

In [13]:
grid = map_input("test2.txt")
dfs = DFS(grid[0])
dfs.process()
path = dfs.get_path()
paths = [[(0, x, y) for x, y in path]]
starts = get_starts(grid)
targets = get_targets(grid)
gui = MapGUI(tk.Tk(), grid, paths, starts, targets)

### UCS

In [14]:
from queue import PriorityQueue

class UCS:
    def __init__(self, grid):
        self.grid = grid
        self.n = len(grid)
        self.m = len(grid[0])
        self.explored = [[False] * self.m for _ in range(self.n)]
        self.directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        self.diagonals = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        self.path = [[(-1, -1)] * self.m for _ in range(self.n)]
        self.distance = [[float('inf')] * self.m for _ in range(self.n)]
        for i in range(self.n):
            for j in range(self.m):
                if grid[i][j] == "A1":
                    self.start = (i, j)
                elif grid[i][j] == "T1":
                    self.target = (i, j)

    def is_valid(self, x, y):
        return 0 <= x < self.n and 0 <= y < self.m and self.grid[x][y] != '-1'

    def process(self):
        queue = PriorityQueue()
        queue.put((0, self.start[0], self.start[1]))
        self.explored[self.start[0]][self.start[1]] = True
        self.distance[self.start[0]][self.start[1]] = 0
        while not queue.empty():
            cost, x, y = queue.get()
            cost = self.distance[x][y]
            if (x, y) == self.target:
                return True

            for dx, dy in self.directions:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny] and self.distance[nx][ny] > cost + 1:
                    queue.put((cost + 1, nx, ny))
                    self.distance[nx][ny] = cost + 1
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
                    
            for dx, dy in self.diagonals:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny] and self.is_valid(x + dx, y) and self.is_valid(x, y + dy) and self.distance[nx][ny] > cost + 1:
                    queue.put((cost + 1, nx, ny))
                    self.distance[nx][ny] = cost + 1
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
        return False
    
    def get_path(self):
        path = []
        x, y = self.target
        while (x, y) != (-1, -1):
            path.append((x, y))
            x, y = self.path[x][y]
        return path[::-1]


In [15]:
grid = map_input("test2.txt")
ucs = UCS(grid[0])
ucs.process()
path = ucs.get_path()
paths = [[(0, x, y) for x, y in path]]
starts = get_starts(grid)
targets = get_targets(grid)
gui = MapGUI(tk.Tk(), grid, paths, starts, targets)

### A*

In [16]:
from queue import PriorityQueue

class AStar:
    def __init__(self, grid):
        self.grid = grid
        self.n = len(grid)
        self.m = len(grid[0])
        self.explored = [[False] * self.m for _ in range(self.n)]
        self.directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        self.diagonals = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        self.path = [[(-1, -1)] * self.m for _ in range(self.n)]
        self.distance = [[float('inf')] * self.m for _ in range(self.n)]
        for i in range(self.n):
            for j in range(self.m):
                if grid[i][j] == "A1":
                    self.start = (i, j)
                elif grid[i][j] == "T1":
                    self.target = (i, j)

    def is_valid(self, x, y):
        return 0 <= x < self.n and 0 <= y < self.m and self.grid[x][y] != '-1'

    def heuristic(self, x, y):
        return max(abs(x - self.target[0]), abs(y - self.target[1]))
    
    def process(self):
        queue = PriorityQueue()
        queue.put((0, self.start[0], self.start[1]))
        self.explored[self.start[0]][self.start[1]] = True
        self.distance[self.start[0]][self.start[1]] = 0
        while not queue.empty():
            cost, x, y = queue.get()
            cost = self.distance[x][y] + self.heuristic(x, y)
            true_cost = self.distance[x][y]
            if (x, y) == self.target:
                return True

            for dx, dy in self.directions:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny] and self.distance[nx][ny] > true_cost + 1:
                    queue.put((cost + 1, nx, ny))
                    self.distance[nx][ny] = true_cost + 1
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
                    
            for dx, dy in self.diagonals:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny] and self.is_valid(x + dx, y) and self.is_valid(x, y + dy) and self.distance[nx][ny] > true_cost + 1:
                    queue.put((cost + 1, nx, ny))
                    self.distance[nx][ny] = true_cost + 1
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
        return False
    
    def get_path(self):
        path = []
        x, y = self.target
        while (x, y) != (-1, -1):
            path.append((x, y))
            x, y = self.path[x][y]
        return path[::-1]


In [17]:
grid = map_input("test2.txt")
a_star = AStar(grid[0])
a_star.process()
path = a_star.get_path()
paths = [[(0, x, y) for x, y in path]]
starts = get_starts(grid)
targets = get_targets(grid)
gui = MapGUI(tk.Tk(), grid, paths, starts, targets)

## Level 2

In [18]:
class Level2Solver:
    def __init__(self, grid):
        self.grid = grid
        self.n = len(grid)
        self.m = len(grid[0])
        self.directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        self.diagonals = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        for i in range(self.n):
            for j in range(self.m):
                if grid[i][j] == "A1":
                    self.start = (i, j)
                elif grid[i][j] == "T1":
                    self.target = (i, j)
        self.distance = {}
        self.path = {}
        self.solution = None
        
    def get_door(self, x, y):
        if self.grid[x][y].startswith("D") and self.grid[x][y] != "DO":
            return self.grid[x][y][1:]
        return None
                
    def is_valid(self, x, y, keys):
        if 0 <= x < self.n and 0 <= y < self.m and self.grid[x][y] != '-1':
            door = self.get_door(x, y)
            if door and door not in keys:
                return False
            return True
        return False
    
    def get_key(self, x, y):
        if self.grid[x][y].startswith("K"):
            return self.grid[x][y][1:]
        return None
    
    def heuristic(self, x, y):
        return max(abs(x - self.target[0]), abs(y - self.target[1]))
    
    def get_path_bfs(self, path, target):
        path_bfs = []
        x, y = target
        while (x, y) != (-1, -1):
            path_bfs.append((x, y))
            x, y = path[x][y]
        path_bfs.pop()
        return path_bfs
    
    def bfs(self, state, cur_distance):
        keys, (x, y) = state
        queue = deque([(x, y)])
        explored = [[False] * self.m for _ in range(self.n)]
        explored[x][y] = True
        distance = [[float('inf')] * self.m for _ in range(self.n)]
        distance[x][y] = 0
        path = [[(-1, -1)] * self.m for _ in range(self.n)]
        new_states = []
        while queue:
            x, y = queue.popleft()
            if (x, y) == self.target:
                new_state = (keys, (x, y))
                if new_state not in self.distance or self.distance[new_state] > cur_distance + distance[x][y]:
                    self.distance[new_state] = cur_distance + distance[x][y]
                    new_states.append(new_state)
                    self.path[new_state] = (state, self.get_path_bfs(path, (x, y)))
                    
            key = self.get_key(x, y)
            if key and key not in keys:
                new_keys = tuple(sorted(keys + (key,)))
                new_state = (new_keys, (x, y))
                if new_state not in self.distance or self.distance[new_state] > cur_distance + distance[x][y]:
                    self.distance[new_state] = cur_distance + distance[x][y]
                    new_states.append(new_state)
                    self.path[new_state] = (state, self.get_path_bfs(path, (x, y)))

            for dx, dy in self.directions:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny, keys) and not explored[nx][ny]:
                    queue.append((nx, ny))
                    explored[nx][ny] = True
                    distance[nx][ny] = distance[x][y] + 1
                    path[nx][ny] = (x, y)
                    
            for dx, dy in self.diagonals:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny, keys) and not explored[nx][ny] and self.is_valid(x + dx, y, keys) and self.is_valid(x, y + dy, keys):
                    queue.append((nx, ny))
                    explored[nx][ny] = True
                    distance[nx][ny] = distance[x][y] + 1
                    path[nx][ny] = (x, y)
                    
        return new_states

    
    def process(self):
        queue = PriorityQueue()
        
        queue.put((0 + self.heuristic(self.start[0], self.start[1]), ((), self.start)))
        self.distance[((), self.start)] = 0
        while not queue.empty():
            cost, state = queue.get()
            true_cost = self.distance[state]
            if true_cost + self.heuristic(state[1][0], state[1][1]) != cost:
                continue
            if state[1] == self.target:
                self.solution = state
                return True
            new_states = self.bfs(state, true_cost)
            for new_state in new_states:
                queue.put((self.distance[new_state] + self.heuristic(new_state[1][0], new_state[1][1]), new_state))
        return False

    def get_path(self):
        if not self.solution:
            return []
        path = []
        state = self.solution
        while state[1] != self.start:
            path.extend(self.path[state][1])
            state = self.path[state][0]
        path.append(self.start)
        return path[::-1]

In [19]:
grid = map_input("test2.txt")
level2_solver = Level2Solver(grid[0])
level2_solver.process()
path = level2_solver.get_path()
paths = [[(0, x, y) for x, y in path]]
starts = get_starts(grid)
targets = get_targets(grid)
gui = MapGUI(tk.Tk(), grid, paths, starts, targets)

## Level 3

In [20]:
class Level3Solver:
    def __init__(self, grid, agents, targets, current_agent):
        self.grid = grid
        self.floors = len(grid)
        self.n = len(grid[0])
        self.m = len(grid[0][0])
        self.directions = [(0, 0, 1), (0, 0, -1), (0, 1, 0), (0, -1, 0)]
        self.diagonals = [(0, 1, 1), (0, 1, -1), (0, -1, 1), (0, -1, -1)]
        self.up = [(1, 0, 0)]
        self.down = [(-1, 0, 0)]
        
        if len(agents) == 0:
            for f in range(self.floors):
                for i in range(self.n):
                    for j in range(self.m):
                        if grid[f][i][j] == "A1":
                            agents.append((f, i, j))
                        elif grid[f][i][j] == "T1":
                            targets.append((f, i, j))
        self.agents = agents
        self.targets = targets
        self.start = agents[current_agent]
        self.target = targets[current_agent]
        
        self.distance = {}
        self.path = {}
        self.solution = None
    
    def is_up(self, f, x, y):
        return 0 <= f < self.floors and 0 <= x < self.n and 0 <= y < self.m and self.grid[f][x][y] == 'UP'
    
    def is_down(self, f, x, y):
        return 0 <= f < self.floors and 0 <= x < self.n and 0 <= y < self.m and self.grid[f][x][y] == 'DO'
    
    def is_valid(self, f, x, y, keys):
        if 0 <= f < self.floors and 0 <= x < self.n and 0 <= y < self.m and self.grid[f][x][y] != '-1':
            door = self.get_door(f, x, y)
            if door and door not in keys:
                return False
            if (f, x, y) in self.agents and (f, x, y) != self.start:
                return False
            return True
        return False
        
    def get_door(self, f, x, y):
        if self.grid[f][x][y].startswith("D") and self.grid[f][x][y] != "DO":
            return self.grid[f][x][y][1:]
        return None
    
    def get_key(self, f, x, y):
        if self.grid[f][x][y].startswith("K"):
            return self.grid[f][x][y][1:]
        return None
    
    def heuristic(self, f, x, y):
        return max(abs(x - self.target[1]), abs(y - self.target[2])) + abs(f - self.target[0])
    
    def get_path_bfs(self, path, target):
        path_bfs = []
        f, x, y = target
        while (f, x, y) != (-1, -1, -1):
            path_bfs.append((f, x, y))
            f, x, y = path[f][x][y]
        path_bfs.pop()
        return path_bfs
    
    def bfs(self, state, cur_distance):
        keys, (f_start, x_start, y_start) = state
        queue = deque([(f_start, x_start, y_start)])
        explored = [[[False] * self.m for _ in range(self.n)] for _ in range(self.floors)]
        explored[f_start][x_start][y_start] = True
        distance = [[[float('inf')] * self.m for _ in range(self.n)] for _ in range(self.floors)]
        distance[f_start][x_start][y_start] = 0
        path = [[[(-1, -1, -1)] * self.m for _ in range(self.n)] for _ in range(self.floors)]
        new_states = []
        while queue:
            f, x, y = queue.popleft()
            if (f, x, y) == self.target:
                new_state = (keys, (f, x, y))
                if new_state not in self.distance or self.distance[new_state] > cur_distance + distance[f][x][y]:
                    self.distance[new_state] = cur_distance + distance[f][x][y]
                    new_states.append(new_state)
                    self.path[new_state] = (state, self.get_path_bfs(path, (f, x, y)))
                    
            key = self.get_key(f, x, y)
            if key and key not in keys:
                new_keys = tuple(sorted(keys + (key,)))
                new_state = (new_keys, (f, x, y))
                if new_state not in self.distance or self.distance[new_state] > cur_distance + distance[f][x][y]:
                    self.distance[new_state] = cur_distance + distance[f][x][y]
                    new_states.append(new_state)
                    self.path[new_state] = (state, self.get_path_bfs(path, (f, x, y)))

            for df, dx, dy in self.directions:
                nf, nx, ny = f + df, x + dx, y + dy
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny]:
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[f][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
            
            for df, dx, dy in self.diagonals:
                nf, nx, ny = f + df, x + dx, y + dy
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny] and self.is_valid(f + df, x + dx, y, keys) and self.is_valid(f + df, x, y + dy, keys):
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[nf][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
            
            if self.is_up(f, x, y):
                nf, nx, ny = f + 1, x, y
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny]:
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[f][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
            
            if self.is_down(f, x, y):
                nf, nx, ny = f - 1, x, y
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny]:
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[f][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
                    
        return new_states

    
    def process(self):
        queue = PriorityQueue()
        
        queue.put((0 + self.heuristic(self.start[0], self.start[1], self.start[2]), ((), self.start)))
        self.distance[((), self.start)] = 0
        while not queue.empty():
            cost, state = queue.get()
            true_cost = self.distance[state]
            if true_cost + self.heuristic(state[1][0], state[1][1], state[1][2]) != cost:
                continue
            if state[1] == self.target:
                self.solution = state
                return True
            new_states = self.bfs(state, true_cost)
            for new_state in new_states:
                queue.put((self.distance[new_state] + self.heuristic(new_state[1][0], new_state[1][1], new_state[1][2]), new_state))
        return False

    def get_path(self):
        if not self.solution:
            return []
        path = []
        state = self.solution
        while state[1] != self.start:
            path.extend(self.path[state][1])
            state = self.path[state][0]
        path.append(self.start)
        return path[::-1]

grid = map_input("test4.txt")
level3_solver = Level3Solver(grid, [], [], 0)
level3_solver.process()
path = level3_solver.get_path()
print(path)

[(0, 0, 3), (0, 0, 2), (0, 0, 1), (0, 0, 2), (0, 0, 3), (0, 0, 4), (0, 1, 4), (0, 2, 4), (0, 3, 3), (0, 3, 2), (0, 3, 1), (1, 3, 1), (1, 3, 2), (1, 3, 3), (1, 4, 4), (1, 4, 3), (1, 4, 2), (1, 4, 1), (0, 4, 1), (0, 4, 2), (0, 3, 3), (0, 2, 4), (0, 1, 4), (0, 0, 4), (0, 0, 3), (0, 0, 2), (0, 1, 1), (0, 1, 0)]


## Level 4

In [21]:
class Level4Solver:
    def __init__(self, grid):
        self.grid = grid
    def process(self):
        
    

SyntaxError: incomplete input (2518281452.py, line 6)